<a href="https://colab.research.google.com/github/DragonFarts/Video-EZY/blob/main/T4_free_colab_of_Stable_Cascade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALL Cascade

In [1]:
%cd /content/
!git lfs install
!git clone https://huggingface.co/spaces/multimodalart/stable-cascade
%cd /content/stable-cascade
!pip install -r /content/stable-cascade/requirements.txt
!pip3 uninstall torch torchvision torchaudio -y
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip uninstall torchdata torchtext -y
!pip install torchdata torchtext

/content
Git LFS initialized.
Cloning into 'stable-cascade'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 205 (delta 119), reused 0 (delta 0), pack-reused 4
Receiving objects: 100% (205/205), 76.53 KiB | 25.51 MiB/s, done.
Resolving deltas: 100% (119/119), done.
Filtering content: 100% (2/2), 88.75 MiB | 27.01 MiB/s, done.


# LOAD Cascade

In [2]:
import torch
import numpy as np
from diffusers import StableCascadeDecoderPipeline, StableCascadePriorPipeline
from diffusers.utils import numpy_to_pil

# Assuming the environment setup and model loading is done as per your script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

# Initialize pipelines (ensure this matches your environment's setup)
prior_pipeline = StableCascadePriorPipeline.from_pretrained("stabilityai/stable-cascade-prior", torch_dtype=dtype).to(device)
decoder_pipeline = StableCascadeDecoderPipeline.from_pretrained("stabilityai/stable-cascade", torch_dtype=dtype).to(device)


/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/models/lora.py:384: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

# Generate a image  (1024x512  requires 13.6gb)

In [10]:
import random
import re


Random_seed = random.randint(0, 6969)

def generate_image(prompt, seed=Random_seed, width=1024, height=1024, prior_guidance_scale=7.5, decoder_guidance_scale=0.0, num_inference_steps=50, num_images_per_prompt=1):
    # Set seed for reproducibility
    Random_seed = random.randint(0, 6969)
    generator = torch.Generator(device=device).manual_seed(seed)

    # Generate with Prior
    prior_results = prior_pipeline(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=prior_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is supported
        generator=generator
    )

    # Extract image embeddings from the generator object
    image_embeddings = None
    for result in prior_results:
        if 'image_embeddings' in result:
            image_embeddings = result['image_embeddings']
            break

    if image_embeddings is None:
        raise ValueError("Failed to generate image embeddings from the prior pipeline")

    # Generate with Decoder
    decoder_output = decoder_pipeline(
        image_embeddings=image_embeddings,
        prompt=prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=decoder_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is correctly applied
        generator=generator,
        output_type="pil"
    )

    return decoder_output.images[0]  # Returns the first (and expected only) generated image

# Example usage
if __name__ == "__main__":
    prompt = "A tabby cat riding on a skateboard down streets of new york"
    Random_seed = random.randint(0, 6969)  # Ensure you have imported random and generated Random_seed

    # Truncate the prompt to a reasonable length and replace spaces with underscores
    safe_prompt = re.sub(r'[^\w\s-]', '', prompt)[:50].strip().replace(' ', '_')

    # Generate the image
    image = generate_image(prompt, num_images_per_prompt=1, seed=Random_seed, width=1024, height=512, prior_guidance_scale=7.5, decoder_guidance_scale=0.0, num_inference_steps=50)

    # Format the filename with the sanitized prompt and seed
    filename = f"/content/{safe_prompt}_{Random_seed}.png"

    # Save the image
    image.save(filename)
    print(f"Image saved as {filename}")
    image.show()

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved as /content/A_tabby_cat_riding_on_a_skateboard_down_streets_of_1290.png
